In [1]:
import arctrl
import os
import frictionless
import re

def validateUsingIsamapData(isamapFilename):
    basePath = os.path.dirname(os.path.abspath(isamapFilename))
    datamap = arctrl.XlsxController.Datamap().from_xlsx_file(isamapFilename)
    files = {}
    patternColumn = re.compile("^col=([0-9]+)+$")
    for datacontext in datamap.DataContexts:
        if datacontext.Format == "text/csv":
            if not datacontext.FilePath in files:
                fileLocation = os.path.join("dataset",datacontext.FilePath)
                if os.path.exists(os.path.join(basePath,fileLocation)):
                    #create automatically resource with schema
                    automaticResource = frictionless.describe(fileLocation, basepath=basePath)
                    #update type to any
                    for i in range(len(automaticResource.schema.fields)):
                        fieldName = automaticResource.schema.fields[i].name
                        automaticResource.schema.fields[i] = frictionless.fields.AnyField(name=fieldName)
                    #update files
                    files[datacontext.FilePath] = {
                        "location": fileLocation,
                        "resource": automaticResource
                    }
                else:
                    raise FileNotFoundError(datacontext.FilePath)
            if datacontext.SelectorFormat == "https://datatracker.ietf.org/doc/html/rfc7111":
                if (colMatch := patternColumn.match(datacontext.Selector)):
                    column = int(colMatch.group(1))
                    value = datacontext.GetValue()
                    fieldName = files[datacontext.FilePath]["resource"].schema.fields[column].name
                    if value.IsAnOntology:
                        if value.Text in ["double","decimal","float"]:                            
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.NumberField(name=fieldName)
                        elif value.Text in ["integer", "long", "int", "short", "byte"]:                            
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.IntegerField(
                                name=fieldName)
                        elif value.Text in ["positiveInteger"]:                            
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.IntegerField(
                                name=fieldName, constraints={"minimum": 1})
                        elif value.Text in ["nonNegativeInteger", "unsignedLong", "unsignedInt", "unsignedShort", "unsignedByte"]:                            
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.IntegerField(
                                name=fieldName, constraints={"minimum": 0})
                        elif value.Text in ["nonPositiveInteger"]:                            
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.IntegerField(
                                name=fieldName, constraints={"maximum": 0})
                        elif value.Text in ["negativeInteger"]:                            
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.IntegerField(
                                name=fieldName, constraints={"maximum": -1})
                        elif value.Text in ["string", "normalizedString", "token"]:
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.StringField(
                                name=fieldName)
                        elif value.Text in ["language"]:
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.StringField(
                                name=fieldName, constraints={"pattern": "[a-zA-Z]{1,8}(-[a-zA-Z0-9]{1,8})*"})
                        elif value.Text in ["boolean"]:
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.BooleanField(name=fieldName)
                        elif value.Text in ["dateTime"]:
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.DateTimeField(name=fieldName)
                        elif value.Text in ["time"]:
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.TimeField(name=fieldName)
                        elif value.Text in ["date"]:
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.DateField(name=fieldName)
                        elif value.Text in ["gYear"]:
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.YearField(name=fieldName)
                        elif value.Text in ["gYearMonth"]:
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.YearmonthField(name=fieldName)
                        elif value.Text in ["duration", "yearMonthDuration", "dayTimeDuration"]:
                            files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.DurationField(name=fieldName)
                        else:
                            raise NotImplementedError(value)
                    elif IsAnInt:
                        files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.IntegerField(name=fieldName)
                    elif IsAFloat:
                        files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.NumberField(name=fieldName)
                    elif value.IsNumerical:
                        files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.NumberField(name=fieldName) 
                    elif value.IsText:
                        files[datacontext.FilePath]["resource"].schema.fields[column] = frictionless.fields.StringField(name=fieldName) 
                    else:
                        raise NotImplementedError(value)
            else:
                raise NotImplementedError(datacontext.SelectorFormat)
        else:
            raise NotImplementedError(datacontext.Format)
    #create response
    response = []
    for id, entry in files.items():
        responseEntry = {"resource": entry["resource"], "validation": entry["resource"].validate()}
        response.append(responseEntry)
    return response

In [2]:
#using ArcPrototype from https://git.nfdi4plants.org/muehlhaus/ArcPrototype/
result = validateUsingIsamapData("ArcPrototype/assays/measurement1/isa.datamap.xlsx")
for entry in result:
    print(entry["resource"].path, entry["validation"].valid, entry["validation"].stats)

dataset/proteomics_result.csv True {'tasks': 1, 'errors': 0, 'warnings': 0, 'seconds': 0.02}
